In [18]:
import pandas as pd
import numpy as np
import torch
import gpytorch
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from torch.utils.data import DataLoader, Dataset
import joblib

In [19]:
import os

date = '24-08-2024'
version = 'v1'
run_id = date + '_' + version

# Create a folder if it doesn't exist
folder_path = f'{run_id}'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA available?:", torch.cuda.is_available())
print("Number of GPUs available:", torch.cuda.device_count())
print("Current GPU:", torch.cuda.current_device())
print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Is CUDA available?: True
Number of GPUs available: 1
Current GPU: 0
GPU Name: NVIDIA GeForce GTX 1660 SUPER


In [21]:
class PollutionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32, device=device)
        self.y = torch.tensor(y.values, dtype=torch.float32, device=device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [22]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())  # RBF kernel is common for continuous data

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize the model and likelihood
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
model = ExactGPModel(None, None, likelihood).to(device)

In [23]:
dataset = pd.read_csv('../Dataset/dataset.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])

dataset = dataset.sample(frac=0.1, random_state=42)

X = dataset.drop(columns=['Date', 'PM2.5', 'Nombre_Estacion', 'Clave_Estacion'])  # Adjust columns as needed
y = dataset['PM2.5']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply quantile transformation to X_train and X_test
# transformer = QuantileTransformer(n_quantiles=1000, output_distribution='normal', random_state=42)
# X_train = transformer.fit_transform(X_train)
# X_test = transformer.transform(X_test)

pollution_dataset_train = PollutionDataset(X_train, y_train)
batch_size = 2048  # Adjust batch size as needed
train_loader = DataLoader(pollution_dataset_train, batch_size=batch_size, shuffle=True)

In [24]:
model.train()
model.set_train_data(pollution_dataset_train.X, pollution_dataset_train.y, strict=False)
likelihood.train()

# Use the Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Marginal Log Likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Training loop
n_epochs = 1000

for epoch in range(n_epochs):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(pollution_dataset_train.X)
    # Calc loss and backprop gradients
    loss = -mll(output, pollution_dataset_train.y)
    loss.backward()
    optimizer.step()

    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        epoch + 1, n_epochs, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))

Iter 1/1000 - Loss: 192.998   lengthscale: 0.694   noise: 0.694
Iter 2/1000 - Loss: 192.709   lengthscale: 0.694   noise: 0.694
Iter 3/1000 - Loss: 192.420   lengthscale: 0.695   noise: 0.695
Iter 4/1000 - Loss: 192.133   lengthscale: 0.695   noise: 0.695
Iter 5/1000 - Loss: 191.847   lengthscale: 0.696   noise: 0.696
Iter 6/1000 - Loss: 191.558   lengthscale: 0.696   noise: 0.696
Iter 7/1000 - Loss: 191.273   lengthscale: 0.697   noise: 0.697
Iter 8/1000 - Loss: 190.988   lengthscale: 0.697   noise: 0.697
Iter 9/1000 - Loss: 190.701   lengthscale: 0.698   noise: 0.698
Iter 10/1000 - Loss: 190.416   lengthscale: 0.698   noise: 0.698
Iter 11/1000 - Loss: 190.132   lengthscale: 0.699   noise: 0.699
Iter 12/1000 - Loss: 189.848   lengthscale: 0.699   noise: 0.699
Iter 13/1000 - Loss: 189.566   lengthscale: 0.700   noise: 0.700
Iter 14/1000 - Loss: 189.282   lengthscale: 0.700   noise: 0.700
Iter 15/1000 - Loss: 189.001   lengthscale: 0.701   noise: 0.701
Iter 16/1000 - Loss: 188.719   len

# Saving the model

In [27]:
# Save the model parameters
joblib.dump(model.state_dict(), f'{run_id}/{run_id}_model_parameters.joblib')
joblib.dump(likelihood.state_dict(), f'{run_id}/{run_id}_likelihood_parameters.joblib')

['24-08-2024_v1/24-08-2024_v1_likelihood_parameters.joblib']

# Load the model

In [28]:
# Load the model parameters
model_parameters = joblib.load(f'{run_id}/{run_id}_model_parameters.joblib')
likelihood_parameters = joblib.load(f'{run_id}/{run_id}_likelihood_parameters.joblib')

# Initialize the model and likelihood with the loaded parameters
model = ExactGPModel(None, None, likelihood).to(device)
model.load_state_dict(model_parameters)
likelihood.load_state_dict(likelihood_parameters)

<All keys matched successfully>

# Evaluate

In [29]:
model.eval()
likelihood.eval()

test_dataset = PollutionDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

all_preds = []
all_actuals = []
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        preds = likelihood(model(batch_x))
        mean = preds.mean
        
        all_preds.append(mean.cpu().numpy())
        all_actuals.append(batch_y.cpu().numpy())

# Flatten the lists of batches
predicted = np.concatenate(all_preds)
actual = np.concatenate(all_actuals)

In [30]:
mse = mean_squared_error(actual, predicted)
r2 = r2_score(actual, predicted)
mae = mean_absolute_error(actual, predicted)

# Revisar como calcular la curva de lorenz

print(f'MSE: {mse:.2f}')
print(f'RMSE: {np.sqrt(mse):.2f}')
print(f'R2: {r2:.2f}')
print(f'MAE: {mae:.2f}')



MSE: 810.47
RMSE: 28.47
R2: -1.00
MAE: 20.11
